In [13]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

In [14]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.1.3
2.5


In [15]:
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport


def sum_over(*args):
    return sum(args)

In [16]:
from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation


ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [19]:
planning_steps

array([0, 1, 2])

In [5]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [6]:
#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
for unc in dike_model.uncertainties:
    print(repr(unc))
    
uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132)
RealParameter('A.1_Bmax', 30, 350)
RealParameter('A.1_pfail', 0, 1)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350)
RealParameter('A.2_pfail', 0, 1)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350)
RealParameter('A.3_pfail', 0, 1)
CategoricalParameter('A.3_Brate', [0, 1, 2])
RealParameter('A.4_Bmax', 30, 350)
RealParameter('A.4_pfail', 0, 1)
CategoricalParameter('A.4_Brate', [0, 1, 2])
RealParameter('A.5_Bmax', 30, 350)
RealParameter('A.5_pfail', 0, 1)
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [7]:
#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))
    
levers = dike_model.levers 

import copy
levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1)
IntegerParameter('0_RfR 1', 0, 1)
IntegerParameter('0_RfR 2', 0, 1)
IntegerParameter('1_RfR 0', 0, 1)
IntegerParameter('1_RfR 1', 0, 1)
IntegerParameter('1_RfR 2', 0, 1)
IntegerParameter('2_RfR 0', 0, 1)
IntegerParameter('2_RfR 1', 0, 1)
IntegerParameter('2_RfR 2', 0, 1)
IntegerParameter('3_RfR 0', 0, 1)
IntegerParameter('3_RfR 1', 0, 1)
IntegerParameter('3_RfR 2', 0, 1)
IntegerParameter('4_RfR 0', 0, 1)
IntegerParameter('4_RfR 1', 0, 1)
IntegerParameter('4_RfR 2', 0, 1)
IntegerParameter('EWS_DaysToThreat', 0, 4)
IntegerParameter('A.1_DikeIncrease 0', 0, 10)
IntegerParameter('A.1_DikeIncrease 1', 0, 10)
IntegerParameter('A.1_DikeIncrease 2', 0, 10)
IntegerParameter('A.2_DikeIncrease 0', 0, 10)
IntegerParameter('A.2_DikeIncrease 1', 0, 10)
IntegerParameter('A.2_DikeIncrease 2', 0, 10)
IntegerParameter('A.3_DikeIncrease 0', 0, 10)
IntegerParameter('A.3_DikeIncrease 1', 0, 10)
IntegerParameter('A.3_DikeIncrease 2', 0, 10)
IntegerParameter('A.4_DikeIncreas

In [8]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=['A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'], function=<function sum_over at 0x00000262E69AFE50>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=['A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'], function=<function sum_over at 0x00000262E69AFE50>)
ScalarOutcome('A.2 Total Costs', variable_name=['A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'], function=<function sum_over at 0x00000262E69AFE50>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=['A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'], function=<function sum_over at 0

In [9]:
#running the model through EMA workbench
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)
 
with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 20 cases completed
[MainProcess/INFO] 40 cases completed
[MainProcess/INFO] 60 cases completed
[MainProcess/INFO] 80 cases completed
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 120 cases completed
[MainProcess/INFO] 140 cases completed
[MainProcess/INFO] 160 cases completed
[MainProcess/INFO] 180 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] experiments finished


In [20]:
experiments.columns

Index(['A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1',
       'discount rate 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0',
       '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0',
       '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2',
       'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2',
       'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2',
       'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2',
       'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2',
       'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2',
       'EWS_DaysToThreat', 'scenario', 'policy', 'model'],
      dtype='object')

In [25]:
experiments[['A.3_Bmax', 'A.3_Brate','A.3_pfail', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', '0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0']]

,A.3_Bmax,A.3_Brate,A.3_pfail,A.3_DikeIncrease 0,A.3_DikeIncrease 1,A.3_DikeIncrease 2,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0
0,67.574232,1,0.935990,3.0,0.0,1.0,0.0,1.0,0.0,0.0
1,89.454513,1.5,0.875831,3.0,0.0,1.0,0.0,1.0,0.0,0.0
2,145.079421,1.5,0.454691,3.0,0.0,1.0,0.0,1.0,0.0,0.0
3,309.497705,1.5,0.160826,3.0,0.0,1.0,0.0,1.0,0.0,0.0
4,150.180720,10,0.259301,3.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
195,78.842132,10,0.594761,10.0,10.0,9.0,0.0,0.0,1.0,1.0
196,162.219591,1,0.105128,10.0,10.0,9.0,0.0,0.0,1.0,1.0
197,331.956374,10,0.309461,10.0,10.0,9.0,0.0,0.0,1.0,1.0
198,314.514026,10,0.032675,10.0,10.0,9.0,0.0,0.0,1.0,1.0


In [10]:
#observing the simulation runs
experiments, outcomes = results
print(outcomes.keys())
experiments

dict_keys(['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs'])


,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,52.0,234.997834,1,0.297260,275.961211,1.5,0.845994,67.574232,1,0.935990,...,4.0,5.0,0.0,1.0,9.0,5.0,3.0,0,0,dikesnet
1,105.0,70.347743,10,0.714210,348.667314,1.5,0.125161,89.454513,1.5,0.875831,...,4.0,5.0,0.0,1.0,9.0,5.0,3.0,1,0,dikesnet
2,40.0,219.523174,1,0.208167,283.624997,10,0.441780,145.079421,1.5,0.454691,...,4.0,5.0,0.0,1.0,9.0,5.0,3.0,2,0,dikesnet
3,123.0,213.246271,1,0.753196,335.207121,10,0.645917,309.497705,1.5,0.160826,...,4.0,5.0,0.0,1.0,9.0,5.0,3.0,3,0,dikesnet
4,65.0,248.437395,10,0.923342,77.475808,1.5,0.397765,150.180720,10,0.259301,...,4.0,5.0,0.0,1.0,9.0,5.0,3.0,4,0,dikesnet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,85.0,288.867689,1,0.589682,227.965860,10,0.556258,78.842132,10,0.594761,...,0.0,10.0,8.0,8.0,5.0,1.0,1.0,45,3,dikesnet
196,39.0,141.962848,1.5,0.383960,273.118687,10,0.689610,162.219591,1,0.105128,...,0.0,10.0,8.0,8.0,5.0,1.0,1.0,46,3,dikesnet
197,2.0,169.927343,10,0.873778,110.761287,1.5,0.491021,331.956374,10,0.309461,...,0.0,10.0,8.0,8.0,5.0,1.0,1.0,47,3,dikesnet
198,5.0,233.439579,10,0.469402,140.354301,1.5,0.513990,314.514026,10,0.032675,...,0.0,10.0,8.0,8.0,5.0,1.0,1.0,48,3,dikesnet


In [26]:
outcomes

{'A.1 Total Costs': array([1.54834840e+08, 1.54834840e+08, 1.55435884e+08, 1.54834840e+08,
        1.54834840e+08, 1.54834840e+08, 1.54834840e+08, 1.54834840e+08,
        1.54834840e+08, 1.54834840e+08, 1.59225787e+08, 1.54834840e+08,
        1.54834840e+08, 1.54834840e+08, 1.60023528e+08, 1.54834840e+08,
        1.18376013e+09, 1.54834840e+08, 1.54834840e+08, 1.54834840e+08,
        1.54834840e+08, 7.87649028e+08, 1.54834840e+08, 1.54834840e+08,
        1.96800393e+08, 1.54834840e+08, 1.54834840e+08, 9.48149317e+08,
        1.54834840e+08, 1.54834840e+08, 1.54834840e+08, 1.54834840e+08,
        1.54834840e+08, 1.54834840e+08, 1.54834840e+08, 1.54834840e+08,
        1.72047931e+08, 1.54834840e+08, 1.54834840e+08, 1.88267217e+08,
        6.92297311e+08, 1.54834840e+08, 1.55813362e+08, 1.54834840e+08,
        1.54834840e+08, 1.54834840e+08, 1.54834840e+08, 1.54834840e+08,
        1.54834840e+08, 1.54834840e+08, 2.89479761e+08, 2.89479761e+08,
        2.89479761e+08, 2.89479761e+08, 2.894

In [11]:
#defining specific policies
#for example, policy 1 is about extra protection in upper boundary
#policy 2 is about extra protection in lower boundary
#policy 3 is extra protection in random locations
from ema_workbench import Policy

policies = [Policy('policy 1', **{'0_RfR 0':1,
                                  '0_RfR 1':1,
                                  '0_RfR 2':1,
                                  'A.1_DikeIncrease 0':5}),
           Policy('policy 2', **{'4_RfR 0':1,
                                  '4_RfR 1':1,
                                  '4_RfR 2':1,
                                  'A.5_DikeIncrease 0':5}),
           Policy('policy 3', **{'1_RfR 0':1,
                                  '2_RfR 1':1,
                                  '3_RfR 2':1,
                                  'A.3_DikeIncrease 0':5})]

In [12]:
#pass the policies list to EMA workbench experiment runs
n_scenarios = 100
with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(n_scenarios,
                                            policies)

[MainProcess/INFO] pool started
[MainProcess/INFO] performing 100 scenarios * 3 policies * 1 model(s) = 300 experiments
[MainProcess/INFO] 30 cases completed
[MainProcess/INFO] 60 cases completed
[MainProcess/INFO] 90 cases completed
[MainProcess/INFO] 120 cases completed
[MainProcess/INFO] 150 cases completed
[MainProcess/INFO] 180 cases completed
[MainProcess/INFO] 210 cases completed
[MainProcess/INFO] 240 cases completed
[MainProcess/INFO] 270 cases completed
[MainProcess/INFO] 300 cases completed
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool
